In [9]:
import multiprocessing.pool
from datetime import datetime
import pandas as pd
import os
import numpy as np
from functools import partial
import dask.dataframe as dd
import dask.multiprocessing
LOB = pd.DataFrame()

In [12]:
laCie = '/media/ak/LaCie/'
trades_quotes_folder = os.path.join(laCie, 'Trades and Quotes Purchased July 2020/')
processed_sampled_data = os.path.join(laCie, 'ProcessedSampledData/')
symbols = os.listdir(trades_quotes_folder)
symbol = 'TY'
symbol_trades_quotes_folder = os.path.join(trades_quotes_folder, symbol)

In [14]:
symbol_files = sorted(os.listdir(symbol_trades_quotes_folder))
symbol_files

['TYH2020.csv',
 'TYH2020.csv.gz',
 'TYH2021.csv.gz',
 'TYM2019.csv.gz',
 'TYM2020.csv.gz',
 'TYU2019.csv.gz',
 'TYU2020.csv.gz',
 'TYZ2019.rar',
 'TYZ2019.txt']

In [15]:
fileidx = 0
idx_symbol_file = os.path.join(symbol_trades_quotes_folder, symbol_files[fileidx])
print(idx_symbol_file)

/media/ak/LaCie/Trades and Quotes Purchased July 2020/TY/TYH2020.csv


In [16]:
df = pd.read_csv(idx_symbol_file)



In [25]:
## checking the df out- not clear it is necessary

types = ['Quote', 'Trade']
df[df['Type']==types[1]] 
# write function that processes all this stuff and saves files in Trades/Quotes similar to the format of everything else

,Date,Time,Type,Price,Volume,Bid Price,Bid Size,Ask Price,Ask Size
11,08/23/2019,11:18:33.017419,Trade,131.59375,1.0,NaN,NaN,NaN,NaN
13,08/27/2019,11:38:51.403200,Trade,132.00000,1.0,NaN,NaN,NaN,NaN
18,08/28/2019,09:05:33.683163,Trade,131.62500,1.0,NaN,NaN,NaN,NaN
41,08/29/2019,12:26:03.843027,Trade,131.78125,1.0,NaN,NaN,NaN,NaN
49,08/29/2019,14:52:08.959326,Trade,132.00000,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
136597138,03/20/2020,10:22:37.647352,Trade,NaN,1.0,NaN,NaN,NaN,NaN
136597669,03/20/2020,10:28:08.491849,Trade,NaN,1.0,NaN,NaN,NaN,NaN
136597792,03/20/2020,10:32:11.083597,Trade,136.00000,1.0,NaN,NaN,NaN,NaN
136598441,03/20/2020,10:49:08.721728,Trade,NaN,1.0,NaN,NaN,NaN,NaN


In [6]:
### conversion of Dates

In [26]:
inputDates = df.Date
date_time_str = inputDates[1]
date_time_str

'07/19/2019'

In [28]:
new_date_time = datetime.strptime(date_time_str, "%m/%d/%Y").strftime("%Y%m%d")

In [29]:
new_date_time

'20190719'

In [30]:
df_per_date = df[df['Date'] == date_time_str]

In [34]:
df

,Date,Time,Type,Price,Volume,Bid Price,Bid Size,Ask Price,Ask Size
0,07/09/2019,11:35:12.666249,Quote,NaN,NaN,127.875000,1.0,NaN,NaN
1,07/19/2019,08:03:20.204314,Quote,NaN,NaN,127.921875,1.0,NaN,NaN
2,07/22/2019,11:59:01.011151,Quote,NaN,NaN,128.015625,5.0,NaN,NaN
3,07/31/2019,12:34:45.703557,Quote,NaN,NaN,125.000000,1.0,NaN,NaN
4,08/04/2019,20:56:45.479555,Quote,NaN,NaN,128.375000,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
136600783,03/20/2020,11:59:33.347390,Quote,NaN,NaN,136.468750,1.0,136.65625,5.0
136600784,03/20/2020,11:59:38.522719,Quote,NaN,NaN,136.468750,1.0,136.71875,1.0
136600785,03/20/2020,11:59:51.675562,Quote,NaN,NaN,136.500000,1.0,136.71875,1.0
136600786,03/20/2020,12:01:44.946946,Quote,NaN,NaN,136.500000,1.0,NaN,NaN


In [13]:
#qt_tmp_df['TradeTime'] = pd.to_datetime(qt_tmp_df.time).values
time_obj =pd.to_datetime(df_per_date.Time)

In [16]:
type(time_obj)

pandas.core.series.Series

In [78]:
# qt_tmp_df['TradeTime'] = pd.to_datetime(qt_tmp_df.time).values
# dfc['Time_of_Sail'] = pd.to_datetime(dfc['Time_of_Sail'],format= '%H:%M:%S' ).dt.time

In [18]:
df_per_date['TradeTime']  = pd.to_datetime(df_per_date.Time).dt.time

/home/ak/anaconda3/envs/newTimeSeries/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
# df_per_date['Date'] = datetime.strptime(df_per_date.Date, "%d/%m/%Y").strftime("%Y%m%d")

In [19]:
bidPrice = df_per_date.filter(like = '-BidPrice')
bidSize = df_per_date.filter(like = '-BidSize')


In [20]:
bidPrice['L1-BidPrice']

0        127.33
1        127.33
2        127.33
3        127.33
4        127.33
          ...  
64032    127.30
64033    127.30
64034    126.42
64035       NaN
64036       NaN
Name: L1-BidPrice, Length: 64037, dtype: float64

In [21]:
LOB['BidPrice'] =(bidSize*bidPrice.values).sum(axis=1)/bidSize.sum(axis=1)
LOB['AskPrice'] =(df_per_date.filter(like = '-AskSize')*df_per_date.filter(like = '-AskPrice').values).sum(axis=1)/df_per_date.filter(like = '-AskSize').sum(axis=1)
# to compute the appropriate size, we take the size at every level of the order book and we create the sume product using size and number of orders at that level of stack, this just creates a summary statistic of the size


In [22]:
df_per_date

,Date,Time,L1-BidPrice,L1-BidSize,L1-BuyNo,L1-AskPrice,L1-AskSize,L1-SellNo,L2-BidPrice,L2-BidSize,...,L9-AskPrice,L9-AskSize,L9-SellNo,L10-BidPrice,L10-BidSize,L10-BuyNo,L10-AskPrice,L10-AskSize,L10-SellNo,TradeTime
0,05/01/2019,08:00:00.169284,127.33,59.0,8.0,127.35,2.0,1.0,127.32,38.0,...,127.43,38.0,9.0,127.24,26.0,10.0,127.44,27.0,6.0,08:00:00.169284
1,05/01/2019,08:00:00.171387,127.33,59.0,8.0,127.35,7.0,2.0,127.32,41.0,...,127.43,41.0,10.0,127.24,29.0,11.0,127.44,30.0,7.0,08:00:00.171387
2,05/01/2019,08:00:00.178396,127.33,59.0,8.0,127.34,19.0,3.0,127.32,41.0,...,127.42,37.0,6.0,127.24,29.0,11.0,127.43,41.0,10.0,08:00:00.178396
3,05/01/2019,08:00:00.190400,127.33,59.0,8.0,127.34,19.0,4.0,127.32,41.0,...,127.42,37.0,6.0,127.24,29.0,11.0,127.43,41.0,10.0,08:00:00.190400
4,05/01/2019,08:00:00.205580,127.33,64.0,9.0,127.34,19.0,4.0,127.32,41.0,...,127.42,37.0,6.0,127.24,29.0,11.0,127.43,41.0,10.0,08:00:00.205580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64032,05/01/2019,18:00:00.128145,127.30,6.0,1.0,NaN,NaN,NaN,127.25,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:00:00.128145
64033,05/01/2019,18:00:00.128145,127.30,6.0,1.0,NaN,NaN,NaN,126.42,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:00:00.128145
64034,05/01/2019,18:00:00.128145,126.42,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:00:00.128145
64035,05/01/2019,18:00:00.128145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:00:00.128145


In [127]:
(df_per_date.filter(like = '-AskSize')*df_per_date.filter(like = '-SellNo').values).sum(axis=1)/df_per_date.filter(like = '-SellNo').sum(axis=1)

0        30.268293
1        32.450980
2        31.957447
3        31.687500
4        32.170213
           ...    
64032          NaN
64033          NaN
64034          NaN
64035          NaN
64036          NaN
Length: 64037, dtype: float64